In [2]:
from datasets import load_dataset
import random
import spacy as sp
import pandas as pd
import numpy as np
import en_core_web_lg
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [52]:
#assuming that both datasets passed in are already dataframes 
def dbindex(target, comparison):
    #load in spaCy pipeline
    nlp = en_core_web_lg.load()
    
    #Pick a random entry from the comparison dataset.
    index = random.randrange(0, len(comparison), 1)

    #Find first entry.
    comparisonEntry = comparison['sentence'][index]
    
    #Vectoritze entry.
    vectorizedComparisonEntry = nlp(comparisonEntry)
    
    #vectorize all entries in target dataset using above pipeline and add to the dataset
    target["vectors"] = target[target.columns[0]].apply(nlp).apply(lambda x : x.vector)
    
    #get list of vectors
    v = target['vectors'].values.tolist()
    
    #run initial k-means
    km = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(v)
    
    #use grid search to optimize number of clusters
    param_grid = {"n_clusters": range(1,5)}
    grid = GridSearchCV(km, param_grid, scoring = 'adjusted_mutual_info_score')
    grid.fit(v)

    #rerun k-means clustering
    km = KMeans(n_clusters=grid.best_params_['n_clusters'], random_state=0, n_init="auto").fit(v)
    
    #find labels for each element, add it to dataframe
    target['clusters'] = km.labels_
    
    #create list for each cluster
    clusters = [[] for _ in range(target['clusters'].nunique())]

    #add sentence to respective cluster
    for index, row in target.iterrows():
        clusters[row[2]].append(row[0])
        
    #initialize list of cosine similarities
    tcossim = []
    
    for cluster in clusters:
        
        #initialize cluster total cosine simliarity
        ccossim = 0
        
        #iterate through each sentence in a cluster (list) 
        for sentence in cluster: 
            vecSentence = nlp(sentence)
            ccossim += vecSentence.similarity(vectorizedComparisonEntry)
        
        tcossim.append(ccossim/len(cluster))
        
    return (sum(tcossim)/len(clusters))

First, government report dataset

In [4]:
base_dataset = load_dataset("ccdv/govreport-summarization")
base_dataset

/home/devam/.local/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


DatasetDict({
    train: Dataset({
        features: ['report', 'summary'],
        num_rows: 17517
    })
    validation: Dataset({
        features: ['report', 'summary'],
        num_rows: 973
    })
    test: Dataset({
        features: ['report', 'summary'],
        num_rows: 973
    })
})

In [59]:
base_dataset.set_format('pandas')
sampleA = pd.DataFrame(base_dataset['train'][:50])

In [60]:
sampleA = pd.DataFrame(bd['report'])

Implicit bias dataset

In [55]:
imbias = load_dataset('henryscheible/implicit_bias')
imbias.set_format(type='pandas')

In [56]:
ib = pd.DataFrame(imbias['train'][:])

In [57]:
ib = pd.DataFrame(ib['sentence'])
ib

,sentence
0,They are OldPeople. Don't ask your grandpa to ...
1,They are Asians. I don't want to invite too ma...
2,They are Queer. Why do all gay men have to be ...
3,They are OldPeople. All Middle Easterners are ...
4,They are Females. She's always so put together...
...,...
1004,They are MiddleEastern. I saw a group of Middl...
1005,"They are Asians. Oh, my grandpa doesn't even k..."
1006,They are Queer. I can't believe she's still tr...
1007,"They are Whites. Oh, you're white, so you prob..."


run db-index calculation for the two

In [61]:
dbindex(sampleA, ib)

/home/devam/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/devam/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 808, in _score
    scores = scorer(estimator, X_test)
TypeError: __call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/home/devam/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/devam/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 808, in _score
    scores = scorer(estimator, X_test)
TypeError: __call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/home/devam/.local/lib

0.7059481813983823